In [144]:
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

import warnings
import numpy as np
import pandas as pd
import yfinance as yf

In [145]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

# Oil Price

In [146]:
CL = yf.download('CL=F', start='2000-08-23')
HO = yf.download('HO=F', start='2000-09-01')
RB = yf.download('RB=F', start='2000-11-01')
BZ = yf.download('BZ=F', start='2007-07-30')
SP = pd.read_csv('Database/oil_price/CL_spot.csv', index_col=0)
BZ_2 = pd.read_csv('Database/oil_price/BZ_future.csv', index_col=0)
SP.sort_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [147]:
SP['price'] = pd.to_numeric(SP['price'], errors='coerce')
SP = SP.dropna()

In [148]:
BZ_2.index = pd.to_datetime(BZ_2.index)
BZ_2_sorted = BZ_2.sort_index().loc[:'2007-07-27', 'Price']
BL = pd.concat([BZ['Adj Close'], BZ_2_sorted], axis=0).sort_index()
BL = pd.DataFrame(data=BL.values, columns=['Adj Close'], index=BL.index)

In [149]:
CL['return'] = CL['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
HO['return'] = HO['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
RB['return'] = RB['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
BL['return'] = BL['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
SP['return'] = SP['price'].pct_change().apply(lambda x: np.log(1 + x))

In [150]:
CL = CL.dropna()
HO = HO.dropna()
RB = RB.dropna()
BL = BL.dropna()
SP = SP.dropna()

In [151]:
CL_vol = CL['return'].rolling(window=5).std()
HO_vol = HO['return'].rolling(window=5).std()
RB_vol = RB['return'].rolling(window=5).std()
BZ_vol = BL['return'].rolling(window=5).std()
SP_vol = SP['return'].rolling(window=5).std()

In [152]:
CL_vol = CL_vol.dropna()
HO_vol = HO_vol.dropna()
RB_vol = RB_vol.dropna()
BZ_vol = BZ_vol.dropna()
SP_vol = SP_vol.dropna()

In [153]:
CL_vol.index = pd.to_datetime(CL_vol.index)
HO_vol.index = pd.to_datetime(HO_vol.index)
RB_vol.index = pd.to_datetime(RB_vol.index)
BZ_vol.index = pd.to_datetime(BZ_vol.index)
SP_vol.index = pd.to_datetime(SP_vol.index)
vol_list = [CL_vol, HO_vol, RB_vol, BZ_vol, SP_vol]

In [119]:
# plt.figure(figsize=(10, 6))
# # plt.plot(CL_vol[:500], label='CL')
# plt.plot(HO_vol[:600], label='HO')
# # plt.plot(NG_vol['return'][:500], label='NG')
# # plt.plot(RB_vol['return'][:500], label='RB')
# plt.plot(BZ_vol[:600], label='BZ')
# # plt.plot(ZL_vol['return'][:500], label='ZL')
# plt.title('Volatility')
# plt.xlabel('Time')
# plt.ylabel('Value')
# plt.legend()
# plt.xticks(CL_vol.index[0:600:50])
# plt.tight_layout()
# plt.show()

# Index

In [6]:
DJ = yf.download('^DJI', start='1986-01-01')

[*********************100%%**********************]  1 of 1 completed


# Treasure Yield

In [224]:
import requests

api_key = 'AJPT9SW0WL4NNYHN'

url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=10year&apikey=demo'

params = {
    'function': 'TREASURY_YIELD',
    'interval': 'daily',
    'maturity': '30year',
    'apikey': api_key
}

r = requests.get(url, params=params)
yield_30 = r.json()

In [225]:
import requests

api_key = 'AJPT9SW0WL4NNYHN'

url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=10year&apikey=demo'

params = {
    'function': 'TREASURY_YIELD',
    'interval': 'daily',
    'maturity': '7year',
    'apikey': api_key
}

r = requests.get(url, params=params)
yield_7 = r.json()

In [226]:
import requests

api_key = 'AJPT9SW0WL4NNYHN'

url = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=monthly&maturity=10year&apikey=demo'

params = {
    'function': 'TREASURY_YIELD',
    'interval': 'daily',
    'maturity': '3month',
    'apikey': api_key
}

r = requests.get(url, params=params)
yield_3m = r.json()

In [227]:
yield_30_df = pd.DataFrame(columns=['date', '30_yield'])
for i in range(len(yield_30['data'])):
    yield_30_df.loc[i, 'date'] = yield_30['data'][i]['date']
    yield_30_df.loc[i, '30_yield'] = yield_30['data'][i]['value']
yield_30_df.set_index(yield_30_df['date'], inplace=True)
yield_30_df = yield_30_df.drop(columns=['date'])

In [228]:
yield_7_df = pd.DataFrame(columns=['date', '7_yield'])
for i in range(len(yield_7['data'])):
    yield_7_df.loc[i, 'date'] = yield_7['data'][i]['date']
    yield_7_df.loc[i, '7_yield'] = yield_7['data'][i]['value']
yield_7_df.set_index(yield_7_df['date'], inplace=True)
yield_7_df = yield_7_df.drop(columns=['date'])

In [229]:
yield_3m_df = pd.DataFrame(columns=['date', '3m_yield'])
for i in range(len(yield_3m['data'])):
    yield_3m_df.loc[i, 'date'] = yield_3m['data'][i]['date']
    yield_3m_df.loc[i, '3m_yield'] = yield_3m['data'][i]['value']
yield_3m_df.set_index(yield_3m_df['date'], inplace=True)
yield_3m_df = yield_3m_df.drop(columns=['date'])

In [230]:
yield_list = [yield_30_df, yield_7_df, yield_3m_df]

In [231]:
for per in yield_list:
    per.index = pd.to_datetime(per.index)

In [232]:
for i, vol in enumerate(vol_list):
    for per in yield_list:
        vol = pd.concat([vol, per], axis=1)
        vol = vol.dropna(subset=['return'])
        vol.replace('.', np.nan, inplace=True)
        vol = vol.ffill()
        vol_list[i] = vol

In [243]:
from sklearn.preprocessing import MinMaxScaler

for i, vol in enumerate(vol_list):
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(vol.iloc[:, 1:])
    data_scaled=pd.DataFrame(data_scaled, index=vol.index)
    vol=pd.concat([vol.iloc[:,0], data_scaled],axis=1)
    vol_list[i] = vol

# Text Data Preprocessing

In [154]:
text_data = pd.read_parquet('Database/newsdata.parquet')

In [156]:
NLP_vec = text_data[['date', 'embeddings']].reset_index()
NLP_vec.index = NLP_vec['date']
NLP_vec = NLP_vec[~NLP_vec.index.duplicated(keep='first')]
NLP_vec = NLP_vec['embeddings']

In [157]:
new_columns = {f'{i}': pd.Series(dtype='object') for i in range(384)}
new_df = pd.DataFrame(new_columns)

for i in range(384):
    new_df[f'{i}'] = NLP_vec.apply(lambda x: f'{x[i]}')

# df = pd.concat([NLP_vec, new_df], axis=1)

In [61]:
for i, vol in enumerate(vol_list):
    vol_list[i] = pd.concat([vol, new_df], axis=1)
    vol_list[i] = vol_list[i].dropna(subset=['return'])
    nan_index = vol_list[i][vol_list[i]['0'].isna()].index
    vol_list[i].loc[nan_index] = 0
    vol_list[i] = vol_list[i].drop(columns=['embeddings'])

KeyboardInterrupt: 

In [158]:
vol_list[4] = pd.concat([vol_list[4], new_df], axis=1)
vol_list[4] = vol_list[4].dropna(subset=['return'])
nan_index = vol_list[4][vol_list[4]['0'].isna()].index
vol_list[4].loc[nan_index, '0':] = 0
# vol_list[4] = vol_list[4].drop(columns=['embeddings'])

In [159]:
vol_list[4]

,return,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
date,,,,,,,,,,,,,,,,,,,,,
1986-01-09,0.018337,-0.04962526261806488,-0.01631726138293743,0.032241448760032654,0.07975346595048904,-0.01664610020816326,0.01926751434803009,-0.019222991541028023,-0.017728831619024277,0.07091069966554642,...,0.02322174608707428,0.010281454771757126,-0.04470190033316612,0.009416778571903706,-0.04469888284802437,-0.03803924843668938,-0.101795993745327,-0.05251132696866989,0.12682756781578064,-0.016033213585615158
1986-01-10,0.018025,-0.0056046294048428535,-0.028814347460865974,0.05053016170859337,-0.010897244326770306,-0.024444127455353737,-0.02611904963850975,-0.09187246114015579,0.053619854152202606,-0.07657802850008011,...,0.19281208515167236,0.00869144219905138,0.020429998636245728,-0.004980586934834719,-0.013956812210381031,0.037604764103889465,-0.05238793045282364,-0.004898854531347752,0.05145702138543129,-0.024697724729776382
1986-01-13,0.014163,-0.02949620969593525,0.023958630859851837,0.0779644250869751,0.04320884868502617,0.05883108079433441,-0.028737040236592293,-0.05733700096607208,0.04011733457446098,-0.013672440312802792,...,-0.005833802744746208,-0.09190331399440765,-0.01094866544008255,0.04017648473381996,-0.044988300651311874,-0.1035151556134224,-0.016279403120279312,-0.06850959360599518,-0.010427573695778847,0.008376100100576878
1986-01-14,0.011612,-0.09731221199035645,-0.06936700642108917,0.1171412542462349,0.11577773839235306,0.049255844205617905,-0.042214665561914444,-0.05348898097872734,0.09646535664796829,-0.004711784888058901,...,0.07969000935554504,-0.03340322896838188,0.02444355934858322,0.027992377057671547,-0.11080018430948257,-0.03989562392234802,-0.0007657308015041053,-0.11314902454614639,-0.06090371683239937,0.02699497528374195
1986-01-15,0.013253,-0.04024816304445267,-0.020085984840989113,0.1438337117433548,0.08445174247026443,0.03432551026344299,0.05127415433526039,-0.11900079250335693,0.08368871361017227,-0.03986898064613342,...,0.042864345014095306,-0.10234757512807846,-0.003282636869698763,0.02656504325568676,-0.05282163619995117,-0.11202266067266464,-0.034381717443466187,-0.045487526804208755,-0.003639000002294779,0.0350753515958786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-12,0.008444,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-03-13,0.014783,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-03-14,0.016125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Files

In [160]:
name_list = ['CL_vol', 'HO_vol', 'RB_vol', 'BZ_vol', 'SP_vol']
for i, vol in enumerate(name_list):
    if i != 4:
        vol_list[i].to_csv(f'Database/future_std/{vol}.csv')
    else:
        vol_list[i].to_csv(f'Database/spot_std/{vol}.csv')